In [1]:
# %pip install sklearn.model
# %pip install xgboost
# %pip install lightgbm

In [109]:
############################################################################################################
## 1. 패키지 import
############################################################################################################
import pandas as pd
import numpy as np
import sqlite3
import sys
from datetime import datetime, timedelta
import time
from pickle import dump, load
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import warnings
import math
warnings.filterwarnings('ignore')

# User Package
from module.logger import log_message
from module.func_mdl_prfmnc import func_mdl_prfmnc
import module.sqlTransaction as sqlT
import importlib
importlib.reload(sqlT)

############################################################################################################
## 2. 초기설정
############################################################################################################
# 경로설정
dir_work = f'c:/Users/user/OneDrive - 파인트리파트너스(주)/movie'
dir_func = f'{dir_work}/src/module'
dir_data = f'{dir_work}/data'
dir_mdl  = f'{dir_work}/data/trn'
dir_prd  = f'{dir_work}/data/prd'
# DB연결
conn = sqlite3.connect(f"{dir_data}/pine_movie.db", isolation_level=None)
cur = conn.cursor()

# 함수 호출
# exec(open(f"{dir_func}/sqlTransaction.py"      , encoding= 'utf-8').read() )
# exec(open(f"{dir_func}/logger.py"              , encoding= 'utf-8').read() )
exec(open(f"{dir_func}/func_mdl_iv_woe.py"              , encoding= 'utf-8').read() )

############################################################################################################
## 3. 파라미터 추출
############################################################################################################
# 입력 파라미터 추출
try :
    bas_ym = sys.argv[1]
    if not bas_ym.isdigit() : raise
except:
    bas_ym = '201206'
    
list_bas_ym   = [datetime.strftime(datetime.strptime(bas_ym, '%Y%m') - relativedelta(months = (i) ), '%Y%m') for i in range(3)]
bas_ym_aft_1m = datetime.strftime(datetime.strptime(bas_ym, '%Y%m') + relativedelta(months = 1 ), '%Y%m')

## 입력데이터 준비
#### 타겟마트 기반 층화추출 방식의 샘플링 - 피쳐마트 결합  

In [3]:
#######################################################################################
# 1. 샘플링
#######################################################################################
# 타겟마트 로드
df_smpl_base = pd.read_sql(f"""
    SELECT
          T1.기준년월 
        , T1.회원번호
        , CASE WHEN 성별 = 'Male'                                           THEN 0  -- 남자
               WHEN 성별 = 'Female'                                         THEN 1  -- 여자
          END                                                                                   AS 성별
        , CASE WHEN 나이 <  20                                              THEN 1
               WHEN 나이 <  30 AND 나이 >= 20                               THEN 2
               WHEN 나이 <  40 AND 나이 >= 30                               THEN 3
               WHEN 나이 <  50 AND 나이 >= 40                               THEN 4
               WHEN 나이 <  60 AND 나이 >= 50                               THEN 5
               WHEN 나이 <  70 AND 나이 >= 60                               THEN 6
               WHEN 나이 <  80 AND 나이 >= 70                               THEN 7
               WHEN 나이 >= 80                                              THEN 8
          END                                                                                  AS 연령대
        , T1.공포_타겟
    FROM TARGET_MART T1
    
    LEFT JOIN CUSTOMER T2
    ON T1.회원번호 = T2.회원번호
    
    WHERE T1.기준년월 BETWEEN {min(list_bas_ym)} and {max(list_bas_ym)}
""", conn)

list_df_smpl = []
for i_bas_ym in list_bas_ym :
    for i_smpl_rt in [1,2] :
        
        # 기준년월별 샘플링 별도 수행 후 통합 방식으로 진행
        df_smpl_tmp_1 = df_smpl_base[df_smpl_base['기준년월'] == int(i_bas_ym)]
        print(f'[LOG] 학습 준비 : 샘플링배수 = {i_smpl_rt}, 건수 = {len(df_smpl_tmp_1)}')
        
        v_cnt_event     = len(df_smpl_tmp_1[df_smpl_tmp_1['공포_타겟'] == 1])
        v_cnt_non_event = len(df_smpl_tmp_1[df_smpl_tmp_1['공포_타겟'] == 0])
        event_rt = v_cnt_event / v_cnt_non_event
        
        # 이벤트/논이벤트 분리 후 논이벤트에 대한 층화추출을 수행
        df_smpl_tmp_2_n = (
            df_smpl_tmp_1[df_smpl_tmp_1['공포_타겟'] != 1]
            .groupby(['성별', '연령대'])
            .sample(
                frac         = event_rt * i_smpl_rt
                , random_state = 0
                , replace      = True # True - 복원추출 / False - 비복원추출 
            )
        )
        
        # 샘플링 배수 컬럼 추가 
        df_smpl_tmp_2_n['샘플링배수'] = i_smpl_rt
        
        # 전체 이벤트 고객 추출 
        df_smpl_tmp_2_y = df_smpl_tmp_1[df_smpl_tmp_1['공포_타겟'] == 1].copy()
        
        # 샘플링 배수 컬럼 추가
        df_smpl_tmp_2_y['샘플링배수'] = i_smpl_rt
        
        # 데이터 통합
        df_smpl_tmp = pd.concat([df_smpl_tmp_2_n, df_smpl_tmp_2_y], ignore_index = True)
        
        # 데이터 Append
        list_df_smpl.append(df_smpl_tmp)

df_smpl = pd.concat(list_df_smpl)


#######################################################################################
# 2. 학습마트 전처리
#######################################################################################
#################################################################
# 2-1. 피쳐마트 로드 및 결합
#################################################################
# 피쳐마트 로드
df_ftr_mart = pd.read_sql(f"""
    SELECT
          *
    FROM INPUT_MART
    WHERE 기준년월 BETWEEN {min(list_bas_ym)} and {max(list_bas_ym)}
""", conn)

df_mart_base = pd.merge(df_smpl[['기준년월', '회원번호', '샘플링배수', '공포_타겟']], df_ftr_mart, on = ['기준년월', '회원번호'], how = 'left')

#################################################################
# 2-2. 결측 처리
#################################################################
list_pk = ['기준년월', '회원번호', '샘플링배수', '공포_타겟']

# 변수 리스트 생성
df_var_list = pd.DataFrame(df_mart_base.dtypes, columns = ["변수유형"]).reset_index().rename(columns = {"index" : "변수명"})

# 연속형 변수 리스트 생성
df_num_list = (
    df_var_list[
            (df_var_list["변수유형"] != 'object')
            & ~(df_var_list["변수명"].isin(list_pk))
    ]
)    
        
# 범주형 변수 리스트 생성
df_char_list = (
    df_var_list[
            (df_var_list["변수유형"] == 'object')
        & ~(df_var_list["변수명"].isin(list_pk))
    ]
)

# 연속형 변수 결측값에 대해 일괄적으로 0으로 대체, 범주형 변수 결측값에 대해 일괄적으로 '기타'로 대체
df_mart_1 = df_mart_base.copy()
df_mart_1[df_num_list["변수명"]]  = df_mart_1[df_num_list["변수명"] ].fillna(0)
df_mart_1[df_char_list["변수명"]] = df_mart_1[df_char_list["변수명"]].fillna('기타')

#################################################################
# 2-3. OneHotEncoding
#################################################################
# 원핫인코딩 적합
oneHotEncdr = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore').fit(df_mart_1[df_char_list["변수명"]])

# 원핫인코딩 적용
tmp_cat = pd.DataFrame(
    oneHotEncdr.transform(df_mart_1[df_char_list["변수명"]]),
    columns=[x.replace(" ", "") for x in oneHotEncdr.get_feature_names_out(df_char_list["변수명"])]
)

# 데이터프레임 초기화
df_onehot_encdr = pd.DataFrame(columns=['변수명', '변수개수', '처리방법'])

# 속성값 개수와 처리방법 저장
for v_onehot in df_char_list['변수명']:
    v_col_cnt = df_mart_1[v_onehot].nunique()
    df_onehot_encdr = pd.concat([df_onehot_encdr, pd.DataFrame({
        '변수명': [v_onehot],
        '변수개수': [v_col_cnt],
        '처리방법': ['원핫인코딩']
    })], ignore_index=True)

# 원핫인코딩 정보로 대체
df_mart_2 = df_mart_1[df_mart_1.columns[~df_mart_1.columns.isin(df_char_list["변수명"])]]

df_mart_2 = pd.concat(
    [df_mart_1[df_mart_1.columns[~df_mart_1.columns.isin(list_pk)]], tmp_cat],
        axis=1
)

with open(f"{dir_mdl}/oneHotEncoder_horror.pkl", "wb") as f: 
    dump(oneHotEncdr, f)

# encoder = load(open(f'{dir_mdl}/oneHotEncoder_horror.pkl', 'rb'))
df_train = pd.concat(
    [ df_mart_1[list_pk], df_mart_2],
        axis=1
)
df_train.to_pickle(f"{dir_mdl}/df_horror_input.pkl")

[LOG] 학습 준비 : 샘플링배수 = 1, 건수 = 1755
[LOG] 학습 준비 : 샘플링배수 = 2, 건수 = 1755
[LOG] 학습 준비 : 샘플링배수 = 1, 건수 = 1720
[LOG] 학습 준비 : 샘플링배수 = 2, 건수 = 1720
[LOG] 학습 준비 : 샘플링배수 = 1, 건수 = 1766
[LOG] 학습 준비 : 샘플링배수 = 2, 건수 = 1766


In [4]:
#######################################################################################
# 3. 모형 생성
#######################################################################################
#################################################################
# 3-1-1. 1차 모형 생성
#################################################################
list_df_valid_prfmnc = []
for i, x_smpl_ratio in enumerate([1,2]) :
    
    df_train_tmp = df_train[df_train['샘플링배수'] == x_smpl_ratio]
    # Train / Valid 분할
    df_X_train, df_X_valid, df_Y_train, df_Y_valid = \
        train_test_split(df_train_tmp[df_train_tmp.columns[~df_train_tmp.columns.isin(list_pk)]], df_train_tmp['공포_타겟'], test_size = 0.2, random_state = 0)

    print(f"""[LOG] 샘플링배수 = {x_smpl_ratio} | Train 데이터 길이 : {len(df_X_train)}""")
    print(f"""[LOG] 샘플링배수 = {x_smpl_ratio} | valid 데이터 길이 : {len(df_X_valid)}""")
    
    # 랜덤포레스트
    mdl_rf = RandomForestClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
    mdl_rf.fit(df_X_train, df_Y_train)
    dump(mdl_rf, open(f'{dir_mdl}/mdl_1st_horror_smpl_{x_smpl_ratio}_rf.pkl', 'wb'))
    
     # XGBOOST모델 학습 및 저장
    mdl_xgb  = XGBClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
    mdl_xgb.fit(df_X_train, df_Y_train)
    dump(mdl_xgb, open(f'{dir_mdl}/mdl_1st_horror_smpl_{x_smpl_ratio}_xgb.pkl', 'wb'))

    # LIGHTGBM모델 학습 및 저장
    mdl_lgb  = LGBMClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
    mdl_lgb.fit(df_X_train, df_Y_train)
    dump(mdl_lgb, open(f'{dir_mdl}/mdl_1st_horror_smpl_{x_smpl_ratio}_lgb.pkl', 'wb'))

    # RF모델 학습결과 생성
    df_Y_valid_rf          = pd.concat([df_Y_valid.reset_index(drop = True), pd.Series([x[1] for x in mdl_rf.predict_proba(df_X_valid)])], axis = 1)
    df_Y_valid_rf.columns  = ['Y_Real', 'Y_Prob'] 

    # xgboost모델 학습결과 생성
    df_Y_valid_xgb         = pd.concat([df_Y_valid.reset_index(drop = True), pd.Series([x[1] for x in mdl_xgb.predict_proba(df_X_valid)])], axis = 1)
    df_Y_valid_xgb.columns = ['Y_Real', 'Y_Prob'] 

    # lightgbm모델 학습결과 생성
    df_Y_valid_lgb         = pd.concat([df_Y_valid.reset_index(drop = True), pd.Series([x[1] for x in mdl_lgb.predict_proba(df_X_valid)])], axis = 1)
    df_Y_valid_lgb.columns = ['Y_Real', 'Y_Prob']
    
    # 검증 데이터셋 성능 지표 산출
    df_valid_prfmnc_rf  = func_mdl_prfmnc(df_Y_valid_rf )
    df_valid_prfmnc_xgb = func_mdl_prfmnc(df_Y_valid_xgb)
    df_valid_prfmnc_lgb = func_mdl_prfmnc(df_Y_valid_lgb)
    
    df_valid_prfmnc_rf['샘플링배수']  = x_smpl_ratio
    df_valid_prfmnc_rf['알고리즘명']  = 'rf'
    df_valid_prfmnc_xgb['샘플링배수'] = x_smpl_ratio
    df_valid_prfmnc_xgb['알고리즘명'] = 'xgb'
    df_valid_prfmnc_lgb['샘플링배수'] = x_smpl_ratio
    df_valid_prfmnc_lgb['알고리즘명'] = 'lgb'
    
    list_df_valid_prfmnc.append(df_valid_prfmnc_rf )
    list_df_valid_prfmnc.append(df_valid_prfmnc_xgb)
    list_df_valid_prfmnc.append(df_valid_prfmnc_lgb)
    
    
# 성능 지표 결과 통합
df_valid_prfmnc = pd.concat(list_df_valid_prfmnc)[['샘플링배수', '알고리즘명','cutoff', 'R1', 'P1', 'RP11', 'RP10', 'RP01', 'RP00', 'precision', 'recall', 'f1score',]]

# 1차 모델 최적 샘플배수 및 알고리즘 추출
v_best_smpl = df_valid_prfmnc[df_valid_prfmnc['f1score'] == df_valid_prfmnc['f1score'].max()]['샘플링배수'].values[0]
v_best_algo = df_valid_prfmnc[df_valid_prfmnc['f1score'] == df_valid_prfmnc['f1score'].max()]['알고리즘명'].values[0]

[LOG] 샘플링배수 = 1 | Train 데이터 길이 : 4317
[LOG] 샘플링배수 = 1 | valid 데이터 길이 : 1080
[LOG] 샘플링배수 = 2 | Train 데이터 길이 : 6476
[LOG] 샘플링배수 = 2 | valid 데이터 길이 : 1619


In [8]:
#################################################################
# 3-1-2. 1차 모형 테스트
#################################################################
#########################################
# 테스트마트 생성
#########################################
df_test = pd.read_sql(f"""
    SELECT 
          T1.*
        , ifnull(T2.공포_타겟, 0)                                   as 공포_타겟
    FROM INPUT_MART T1
    LEFT JOIN TARGET_MART T2
    ON 
        T1.기준년월 = T2.기준년월
    AND T1.회원번호 = T2.회원번호
    WHERE T1.기준년월 = {bas_ym_aft_1m}
""", conn)

#########################################
# 결측 처리
#########################################
list_pk = ['기준년월', '회원번호', '샘플링배수', '공포_타겟']

# 변수 리스트 생성
df_var_list = pd.DataFrame(df_test.dtypes, columns = ["변수유형"]).reset_index().rename(columns = {"index" : "변수명"})

# 연속형 변수 리스트 생성
df_num_list = (
    df_var_list[
            (df_var_list["변수유형"] != 'object')
            & ~(df_var_list["변수명"].isin(list_pk))
    ]
)    
        
# 범주형 변수 리스트 생성
df_char_list = (
    df_var_list[
            (df_var_list["변수유형"] == 'object')
        & ~(df_var_list["변수명"].isin(list_pk))
    ]
)

# 연속형 변수 결측값에 대해 일괄적으로 0으로 대체, 범주형 변수 결측값에 대해 일괄적으로 '기타'로 대체
df_test_mart_1 = df_test.copy()
df_test_mart_1[df_num_list["변수명"]]  = df_test_mart_1[df_num_list["변수명"] ].fillna(0)
df_test_mart_1[df_char_list["변수명"]] = df_test_mart_1[df_char_list["변수명"]].fillna('기타')

#########################################
# OneHotEncoding
#########################################
# 원핫인코더 로드
oneHotEncdr = load(open(f"{dir_mdl}/oneHotEncoder_horror.pkl", 'rb'))

# 원핫인코딩 적용
tmp_cat = pd.DataFrame(
    oneHotEncdr.transform(df_test_mart_1[df_char_list["변수명"]]),
    columns=[x.replace(" ", "") for x in oneHotEncdr.get_feature_names_out(df_char_list["변수명"])]
)
if len(tmp_cat.columns) > 0 : 
    df_test_mart_2 = pd.concat(
        [df_test_mart_1[df_test_mart_1[df_test_mart_1.columns[~df_test_mart_1.columns.isin(df_char_list['변수명'].to_list())]].reset_index(drop=True), tmp_cat.reset_index(drop=True)]]
    )
else : 
    df_test_mart_2 = df_test_mart_1.copy()
    
#########################################
# 1차 모형 테스트 시작
#########################################
# 테스트마트 데이터를 X와 Y로 분리
df_test_1st_X = df_test_mart_2.drop(columns = ['기준년월', '회원번호', '공포_타겟'])
df_test_1st_Y = df_test_mart_2['공포_타겟'].astype(int)

list_df_test_prfmnc = []
for i, smpl_ratio in enumerate([1,2]) :
    for i, algo in enumerate(['rf', 'xgb', 'lgb']) :
        print(f'[LOG] 샘플배수 = {smpl_ratio} | 알고리즘 = {algo}')
        mdl = pd.read_pickle(f'{dir_mdl}/mdl_1st_horror_smpl_{smpl_ratio}_{algo}.pkl')
        df_test_scr =  pd.concat([df_test_1st_Y.reset_index(drop = True), pd.Series([x[1] for x in mdl .predict_proba(df_test_1st_X)])], axis = 1)
        df_test_scr.columns = ['Y_Real', 'Y_Prob']
        
        # 테스트 성능 지표 산출
        df_test_1st_prfmnc_tmp  = func_mdl_prfmnc(df_test_scr)
        
        # 모형 라벨링 정보 추가
        df_test_1st_prfmnc_tmp['알고리즘명'] = algo
        df_test_1st_prfmnc_tmp['샘플링배수'] = smpl_ratio

        list_df_test_prfmnc.append(df_test_1st_prfmnc_tmp)

# 성능 지표 결과 통합
df_test_1st_prfmnc = pd.concat(list_df_test_prfmnc)[['샘플링배수','알고리즘명','cutoff', 'R1', 'P1', 'RP11', 'RP10', 'RP01', 'RP00', 'precision', 'recall', 'f1score',]]

# 1차 모형 테스트셋 최적 알고리즘 추출
v_test_best_algo = df_test_1st_prfmnc[df_test_1st_prfmnc['f1score'] == df_test_1st_prfmnc['f1score'].max()]['알고리즘명'].values[0]
v_test_best_smpl = df_test_1st_prfmnc[df_test_1st_prfmnc['f1score'] == df_test_1st_prfmnc['f1score'].max()]['샘플링배수'].values[0]


[LOG] 샘플배수 = 1 | 알고리즘 = rf
[LOG] 샘플배수 = 1 | 알고리즘 = xgb
[LOG] 샘플배수 = 1 | 알고리즘 = lgb
[LOG] 샘플배수 = 2 | 알고리즘 = rf
[LOG] 샘플배수 = 2 | 알고리즘 = xgb
[LOG] 샘플배수 = 2 | 알고리즘 = lgb


In [10]:
#################################################################
# 3-2-1. 변수선택
#################################################################
# IV WOE 산출을 위한 입력마트 생성
df_iv_base = pd.read_sql(f""" 
    SELECT 
          T1.*
        , IFNULL(T2.공포_타겟, 0)                   AS 공포_타겟
    FROM INPUT_MART T1
    
    LEFT JOIN TARGET_MART T2
    ON 
        T1.기준년월 = T2.기준년월
    AND T1.회원번호 = T2.회원번호
    
    WHERE T1.기준년월 BETWEEN {min(list_bas_ym)} and {max(list_bas_ym)}
""", conn)

df_iv_base = df_iv_base.rename(columns = {'공포_타겟' : 'y'})
df_iv_res = func_mdl_iv_woe(df_iv_base,  ['기준년월', '회원번호', 'y'])

# 1차 채택 모형의 변수중요도 추출
mdl_best_algo_1st = pd.read_pickle(f'{dir_mdl}/mdl_1st_horror_smpl_{v_test_best_smpl}_{v_test_best_algo}.pkl')  
df_ftr_imp = pd.DataFrame({
      'varNm'       : mdl_best_algo_1st.feature_names_in_
    # , 'algorithm' : 'rf'
    , '변수중요도'    : mdl_best_algo_1st.feature_importances_/mdl_best_algo_1st.feature_importances_.sum()
}).sort_values(['변수중요도'], ascending = False)

# 영향인자분석 결과 통합
df_factor = pd.merge(df_ftr_imp, df_iv_res.groupby(['varNm']).max()['iv_sum'].round(5).reset_index(), on = 'varNm', how = 'left')

###############################################
# 영향인자분석 결과를 반영한 변수선택 결과 추출 
#  - IV 상위 60% 
#  - 연관장르
###############################################
# IV 상위 추출
df_ftr_selection = df_factor.sort_values(by = ['iv_sum'], ascending = False)[:int(len(df_factor) * 0.6)]

# 연관장르만 추출
list_unrelated_genre = ['드라마' ,'코미디' ,'전쟁' ,'로맨스' ,'가족' ,'애니메이션' ,'스포츠' ,'전기' ,'뮤지컬' ,'음악' ,'단편극' ,'역사' ,'서부극' ,'다큐멘터리']
list_ftr_selection_2 = [x for x in df_ftr_selection['varNm'].to_list() 
    if   any(key in x for key in list_unrelated_genre)  # 비연관 장르 제외
      or ('최근' not in x)                              # 행동정보 이외 컬럼 
]
list_ftr_selection = df_ftr_selection[df_ftr_selection['varNm'].isin(list_ftr_selection_2)]['varNm'].to_list()

# 변수선택 결과 저장
with open(f'{dir_mdl}/list_horror_ftr_selection.pkl', 'wb') as f:
    dump(list_ftr_selection, f)

[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 0 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 10 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 20 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 30 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 40 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 50 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 60 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 70 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 80 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 90 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 100 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 110 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 120 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 130 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 140 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 150 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 160 / 180
[LOG] func_mdl_pre_iv > 연속형 변수 Binning : 170 / 180
[LOG] func_mdl_pre_iv > IV 산출 : 0 / 180
[LOG] func_mdl_pre_iv > IV 산출 : 10 / 180
[LOG] func_m

In [33]:
#################################################################
# 3-3-1. 2차 모형 생성
#################################################################
# 2차모형 조건 추출
#  - v_test_best_algo  #최적알고리즘
#  - v_test_best_smpl  #최적샘플배수
#  - list_ftr_selection #변수선택결과

# 학습 데이터 로드 및 조건반영
df_train_2nd = df_train[(df_train['샘플링배수'] == v_test_best_smpl)][list_pk + list_ftr_selection]
list_pk = ['기준년월', '회원번호', '공포_타겟']

#########################################
# 2차 모형 학습
#########################################
# Train / Valid 분할
df_X_train, df_X_valid, df_Y_train, df_Y_valid = \
    train_test_split(df_train_2nd[df_train_2nd.columns[~df_train_2nd.columns.isin(list_pk)]], df_train_2nd['공포_타겟'], test_size = 0.2, random_state = 0)

print(f'[LOG] 2차 모형 학습 | 최적알고리즘 = {v_test_best_algo} |')

if v_test_best_algo == 'rf' :
    mdl_2nd = RandomForestClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
elif v_test_best_algo == 'xgb' :
    mdl_2nd = XGBClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
elif v_test_best_algo == 'lgb' :
    mdl_2nd = LGBMClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
   
mdl_2nd.fit(df_X_train, df_Y_train)
dump(mdl_2nd, open(f'{dir_mdl}/mdl_horror_smpl_{v_test_best_smpl}_{v_test_best_algo}_2nd.pkl', 'wb'))

# 모형 검증
df_Y_valid          = pd.concat([df_Y_valid.reset_index(drop = True), pd.Series([x[1] for x in mdl_2nd.predict_proba(df_X_valid)])], axis = 1)
df_Y_valid.columns  = ['Y_Real', 'Y_Prob'] 
df_valid_prfmnc_2nd = func_mdl_prfmnc(df_Y_valid_rf)

#########################################
# 2차 모형 테스트 시작
#########################################
# 테스트마트 데이터를 X와 Y로 분리
df_test_2nd_X = df_test_1st_X.copy()[list_ftr_selection]
df_test_2nd_Y = df_test_1st_Y.copy()

# 2차 모형 테스트 스코어 산출
df_test_2nd_scr =  pd.concat([df_test_2nd_Y.reset_index(drop = True), pd.Series([x[1] for x in mdl_2nd.predict_proba(df_test_2nd_X)])], axis = 1)
df_test_2nd_scr.columns = ['Y_Real', 'Y_Prob']

# 2차 모형 테스트 성능 지표 산출
df_test_2nd_prfmnc  = func_mdl_prfmnc(df_test_2nd_scr)

# 1차 2차 성능 비교
df_1st = df_test_1st_prfmnc[df_test_1st_prfmnc['f1score'] == df_test_1st_prfmnc[(df_test_1st_prfmnc['알고리즘명'] == v_best_algo) & (df_test_1st_prfmnc['샘플링배수'] == v_best_smpl)]['f1score'].max()].drop(columns=['샘플링배수', '알고리즘명'])
df_2nd = df_test_2nd_prfmnc[df_test_2nd_prfmnc['f1score'] == df_test_2nd_prfmnc['f1score'].max()]
df_1st['모형구분'] = '1차결과'
df_2nd['모형구분'] = '2차결과'
df_1st_2nd_comparison = pd.concat([df_1st, df_2nd])
    

[LOG] 2차 모형 학습 | 최적알고리즘 = rf |


In [63]:
#########################################
# 파생변수 생성
#########################################
list_drv_bas_ym = [datetime.strftime(datetime.strptime(bas_ym, '%Y%m') - relativedelta(months = (i-1) ), '%Y%m') for i in range(4)]
list_df_drv_ftr = []

# 최근 3개월, 6개월 간 연관 장르 행동정보 추가
for i, x_bas_ym in enumerate(list_drv_bas_ym) :
  x_bas_ym_bfr_2m = datetime.strftime(datetime.strptime(x_bas_ym, '%Y%m') - relativedelta(months = 2 ), '%Y%m')
  x_bas_ym_bfr_5m = datetime.strftime(datetime.strptime(x_bas_ym, '%Y%m') - relativedelta(months = 5 ), '%Y%m')

  df_drv_ftr_tmp = pd.read_sql(f"""
    select
          {x_bas_ym}                                                                                                           as 기준년월
        , T1.회원번호

        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  7      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_액션_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  7      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_액션_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  7      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_액션_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  7      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_액션_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  7      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_액션_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  9      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_스릴러_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  9      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_스릴러_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  9      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_스릴러_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  9      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_스릴러_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  9      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_스릴러_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 45      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_공상과학_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 45      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_공상과학_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 45      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_공상과학_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 45      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_공상과학_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 45      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_공상과학_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  8      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_범죄_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  8      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_범죄_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  8      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_범죄_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  8      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_범죄_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 =  8      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_범죄_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 12      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_판타지_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 12      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_판타지_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 12      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_판타지_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 12      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_판타지_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 12      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_판타지_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 11      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_모험_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 11      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_모험_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 11      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_모험_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 11      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_모험_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 11      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_모험_구매횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 20      and T2.행동번호 = 5         then 1 else 0 end)         as 최근6개월_미스테리_탐색횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 20      and T2.행동번호 = 1         then 1 else 0 end)         as 최근6개월_미스테리_평가횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 20      and T2.행동번호 = 2         then 1 else 0 end)         as 최근6개월_미스테리_시청완료횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 20      and T2.행동번호 = 4         then 1 else 0 end)         as 최근6개월_미스테리_시청시작횟수
        , sum(case WHEN T2.기간 = '최근6개월' and  T2.장르번호 = 20      and T2.행동번호 = 11        then 1 else 0 end)         as 최근6개월_미스테리_구매횟수
        
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  7      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_액션_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  7      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_액션_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  7      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_액션_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  7      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_액션_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  7      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_액션_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  9      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_스릴러_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  9      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_스릴러_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  9      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_스릴러_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  9      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_스릴러_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  9      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_스릴러_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 45      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_공상과학_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 45      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_공상과학_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 45      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_공상과학_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 45      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_공상과학_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 45      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_공상과학_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  8      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_범죄_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  8      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_범죄_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  8      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_범죄_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  8      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_범죄_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 =  8      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_범죄_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 12      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_판타지_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 12      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_판타지_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 12      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_판타지_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 12      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_판타지_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 12      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_판타지_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 11      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_모험_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 11      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_모험_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 11      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_모험_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 11      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_모험_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 11      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_모험_구매횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 20      and T2.행동번호 = 5         then 1 else 0 end)         as 최근3개월_미스테리_탐색횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 20      and T2.행동번호 = 1         then 1 else 0 end)         as 최근3개월_미스테리_평가횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 20      and T2.행동번호 = 2         then 1 else 0 end)         as 최근3개월_미스테리_시청완료횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 20      and T2.행동번호 = 4         then 1 else 0 end)         as 최근3개월_미스테리_시청시작횟수
        , sum(case WHEN T2.기간 = '최근3개월' and  T2.장르번호 = 20      and T2.행동번호 = 11        then 1 else 0 end)         as 최근3개월_미스테리_구매횟수

    -- 대상 고객 : 최근 3개월 활성고객
    FROM (
        SELECT
                회원번호
        FROM MOVIE_FACT
        WHERE 기준년월 BETWEEN {x_bas_ym_bfr_2m} and {x_bas_ym}
        GROUP BY 회원번호
    ) T1

    LEFT JOIN (
        SELECT
              기준년월 
            , CASE WHEN a01.기준년월 BETWEEN {x_bas_ym_bfr_2m} and {x_bas_ym}            then  '최근3개월'
                  ELSE                                                                         '최근6개월'
              END                                                                                       as 기간
            , a01.회원번호
            , a02.장르번호
            , a01.행동번호
        FROM MOVIE_FACT a01

        ----------------------------------------------------------------
        -- 장르 매핑
        ----------------------------------------------------------------
        LEFT JOIN (
            SELECT
                  영화번호
                , 장르번호
            FROM MOVIE_GENRE

            --------------------------------------------
            -- 중복 장르 번호 제거
            --------------------------------------------
            EXCEPT
            SELECT
                  b01.영화번호
                , b02.장르번호
            FROM MOVIE b01

            LEFT JOIN MOVIE_GENRE b02 -- 영화 장르 매핑
            ON b01.영화번호 = b02.영화번호

            LEFT JOIN GENRE b03 -- 장르명 매핑
            ON b02.장르번호 = b03.장르번호

            GROUP BY
                  b01.영화번호
                , b03.장르명
            HAVING COUNT(1) > 1  -- 중복 존재 장르번호 추출
        ) a02
        ON a01.영화번호 = a02.영화번호

        WHERE a01.기준년월 BETWEEN {x_bas_ym_bfr_5m} and {bas_ym}
        GROUP BY 
            a01.기준년월
          , a01.회원번호
          , a02.장르번호
          , a01.행동번호
    ) T2
    ON T1.회원번호 = T2.회원번호

    group by t1.회원번호
  """
  , conn)
  
  list_df_drv_ftr.append(df_drv_ftr_tmp)

df_drv_ftr = pd.concat(list_df_drv_ftr)
df_train_3rd = pd.merge(df_train_2nd, df_drv_ftr, on = ['기준년월', '회원번호'], how = 'left' ).fillna(0)

In [58]:
#########################################
# 3차 모형 학습
#########################################
# Train / Valid 분할
df_X_train = df_train_3rd[df_train_3rd.columns[~df_train_3rd.columns.isin(list_pk)]]
df_Y_train = df_train_3rd['공포_타겟']

print(f'[LOG] 3차 모형 학습 | 최적알고리즘 = {v_test_best_algo} |')

if v_test_best_algo == 'rf' :
    mdl_3rd = RandomForestClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
elif v_test_best_algo == 'xgb' :
    mdl_3rd = XGBClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)
elif v_test_best_algo == 'lgb' :
    mdl_3rd = LGBMClassifier(n_estimators = 100, max_depth = 10, random_state=0, verbose = 0)

mdl_3rd.fit(df_X_train, df_Y_train)
dump(mdl_3rd, open(f'{dir_mdl}/mdl_3rd_horror_smpl_{v_test_best_smpl}_{v_test_best_algo}.pkl', 'wb'))


[LOG] 3차 모형 학습 | 최적알고리즘 = rf |


In [113]:
#########################################
# 3차 모형 테스트 시작
#########################################
# 테스트마트 데이터를 X와 Y로 분리
df_test_3rd = pd.merge(df_test_mart_2[list_pk + list_ftr_selection], df_drv_ftr, on = ['기준년월', '회원번호'], how = 'left')
df_test_3rd_X = df_test_3rd[df_test_3rd.columns[~df_test_3rd.columns.isin(list_pk)]]
df_test_3rd_Y = df_test_3rd['공포_타겟']

# 3차 모형 테스트 스코어 산출
df_test_3rd_scr =  pd.concat([df_test_3rd_Y.reset_index(drop = True), pd.Series([x[1] for x in mdl_3rd.predict_proba(df_test_3rd_X)])], axis = 1)
df_test_3rd_scr.columns = ['Y_Real', 'Y_Prob']

# 2차 모형 테스트 성능 지표 산출
df_test_3rd_prfmnc  = func_mdl_prfmnc(df_test_3rd_scr)

# 1차 2차 성능 비교
df_2nd = df_test_2nd_prfmnc[df_test_2nd_prfmnc['f1score'] == df_test_2nd_prfmnc['f1score'].max()]
df_3rd = df_test_3rd_prfmnc[df_test_3rd_prfmnc['f1score'] == df_test_3rd_prfmnc['f1score'].max()]
df_2nd['모형구분'] = '2차결과'
df_3rd['모형구분'] = '3차결과'
df_2nd_3rd_comparison = pd.concat([df_2nd, df_3rd])

display(df_2nd_3rd_comparison)

#########################################
# 챔피언 모형 선정 및 저장
#########################################
chmp_mdl    = mdl_3rd if df_test_2nd_prfmnc['f1score'].max() < df_test_3rd_prfmnc['f1score'].max() else mdl_2nd
chmp_prfmnc = df_test_3rd_prfmnc if df_test_2nd_prfmnc['f1score'].max() < df_test_3rd_prfmnc['f1score'].max() else df_test_2nd_prfmnc
with open(f"{dir_prd}/mdl_horror.pkl", "wb") as f: 
    dump(chmp_mdl, f)
with open(f"{dir_prd}/df_prfmnc.pkl", "wb") as f: 
    dump(chmp_prfmnc, f)
    
# 최종 변수 리스트 저장
list_ftr_fin = list_ftr_selection + [x for x in df_drv_ftr.columns.tolist() if x not in list_pk]
with open(f"{dir_prd}/list_horror_ftr.pkl", "wb") as f: 
    dump(list_ftr_fin, f)
    
# 원핫인코더 운영 경로 저장
with open(f"{dir_prd}/oneHotEncoder_horror.pkl", "wb") as f: 
    dump(oneHotEncdr, f)

,cutoff,R1,P1,RP11,RP10,RP01,RP00,precision,recall,f1score,모형구분
0,54,884,973,1712,169,80,804,0.955357,0.910154,0.932208,2차결과
0,47,884,979,1718,163,68,816,0.961926,0.913344,0.937006,3차결과
